# 0.0 - Ridge Linear Regression

Os parâmetros utilizados para treinar o algoritmo **Ridge Linear Regression** serão a penalidade de regularização `alpha` e o número máximo de iterações, `max_iter`.</br>Os valores padrão (*default*) nesse caso são **alpha=1.0** e **max_iter=None**, segundo a documentação oficial disponível em:</br></br>https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html

# 1.0 - Importando bibliotecas

In [40]:
import pandas as pd
import numpy as np
from sklearn import linear_model as lm
import matplotlib.pyplot as plt

from itertools import product

from sklearn.metrics import r2_score, mean_squared_error, root_mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

# 2.0 - Carregando os dados

In [41]:
# Treino
X_train = pd.read_csv('train/X_training.csv')
y_train = pd.read_csv('train/y_training.csv')

# Validação
X_val = pd.read_csv('validation/X_validation.csv')
y_val = pd.read_csv('validation/y_val.csv')

# Teste
X_test = pd.read_csv('test/X_test.csv')
y_test = pd.read_csv('test/y_test.csv')

# 3.0 - Funções

In [42]:
def metrics(y, y_pred, step, algorithm):
    """
    Calcula várias métricas de avaliação de um modelo de regressão e as retorna em um DataFrame.

    Parâmetros:
    -----------
    y : array-like
        Valores da variável resposta.
    
    y_pred : array-like
        Valores preditos pelo modelo.
    
    step : str
        Etapa do processo de modelagem (ex: 'treino', 'validação', 'teste').
    
    algorithm : str
        Nome do algoritmo usado.

    Retorno:
    --------
    df_metrics : pandas.DataFrame
        DataFrame contendo as métricas calculadas:
        - 'Algorithm': Nome do algoritmo.
        - 'Step': Etapa do processo (treino, validação ou teste).
        - 'R2': Coeficiente de determinação.
        - 'MSE': Erro quadrático médio.
        - 'RMSE': Raiz do erro quadrático médio.
        - 'MAE': Erro absoluto médio.
        - 'MAPE': Erro percentual absoluto médio.
    """
    # Calcula as métricas
    r2 = r2_score(y, y_pred)
    mse = mean_squared_error(y, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y, y_pred)
    mape = mean_absolute_percentage_error(y, y_pred)

    # Cria o DataFrame com as métricas
    df_metrics = pd.DataFrame({   
        'Algorithm': [algorithm],
        'Step': [step],
        'R2': [np.round(r2, 4)],
        'MSE': [np.round(mse, 4)],
        'RMSE': [np.round(rmse, 4)],
        'MAE': [np.round(mae, 4)],
        'MAPE': [np.round(mape, 4)]
    })
    
    return df_metrics

def exporta_excel(nome_algoritmo):
    """
    Exporta os DataFrames de métricas para arquivos Excel em diferentes pastas.

    Args:
        nome_algoritmo (str): Nome do algoritmo utilizado.

    Returns:
        None
    """
    etapas = ['train', 'validation', 'test']
    df_list = [df_metrics_train, df_metrics_val, df_metrics_test]

    for etapa, df in zip(etapas, df_list):
        df.to_excel(f'{etapa}/metrics/metrics_{etapa}_{nome_algoritmo}.xlsx', index=False)

# 4.0 - Ajustando os dados

In [43]:
# transformação do formato DataFrame para Series, essencialmente um array unidimensional
y_train, y_val, y_test = [ df.loc[:, 'song_popularity'] for df in [y_train, y_val, y_test] ]

# 5.0 - Desempenho do modelo

## 5.1 - Dados de treino

In [44]:
# definição do modelo
# Ridge - L2 (ameniza os coeficientes da equação linear)
ridge_train = lm.Ridge( random_state = 0 )

# treinamento do algoritmo.
ridge_train.fit( X_train, y_train )

# Predição sobre os dados de treino
y_pred_train = ridge_train.predict( X_train )

# Métricas
df_metrics_train = metrics(y_train, y_pred_train, 'Train', 'Ridge Linear Regression')
df_metrics_train

,Algorithm,Step,R2,MSE,RMSE,MAE,MAPE
0,Ridge Linear Regression,Train,0.0461,455.9964,21.3541,16.9983,8.6534


## 5.2 - Dados de validação

In [46]:
# definição do modelo
ridge_val = lm.Ridge( random_state = 0 )

# treinamento do algoritmo. 
ridge_val.fit(X_train, y_train)

# Predição sobre os dados de validacao
y_pred_val = ridge_val.predict(X_val)

# Métricas
df_metrics_val = metrics(y_val, y_pred_val, 'Validation', 'Ridge Linear Regression')
df_metrics_val

,Algorithm,Step,R2,MSE,RMSE,MAE,MAPE
0,Ridge Linear Regression,Validation,0.0399,458.4455,21.4113,17.0395,8.6824


## 5.3 - Ajuste fino dos hiperparâmetros (Etapa de *Fine Tuning*)

In [47]:
# Definir intervalo de parâmetros
alpha_array = np.arange(0.5, 11, 0.5)
max_iter_list = [100, 500, 1000, 1500]

# Criar DataFrame para armazenar resultados
metrics_df = pd.DataFrame(columns=['alpha', 'max_iter', 'R2', 'MSE', 'RMSE', 'MAE', 'MAPE'])

# Usar 'itertools.product' para combinar os valores dos hiper-parâmetros
for m, n in product(alpha_array, max_iter_list):
    # Definir e treinar o modelo 
    model = lm.Ridge(alpha=m, max_iter=n, random_state=0)
    model.fit(X_train, y_train)

    # Predição sobre os dados de validação
    y_pred = model.predict(X_val)
    
    # Calcular métricas de performance
    r2_tuning  = r2_score( y_val, y_pred )
    mse_tuning  = mean_squared_error( y_val, y_pred )
    rmse_tuning = root_mean_squared_error( y_val, y_pred )
    mae_tuning  = mean_absolute_error( y_val, y_pred )
    mape_tuning = mean_absolute_percentage_error( y_val, y_pred )
    
    # Armazenar os resultados em uma nova linha como DataFrame
    new_row = pd.DataFrame({
        'alpha': [m],
        'max_iter': [n],
        'R2': [r2_tuning],
        'MSE': [mse_tuning],
        'RMSE': [rmse_tuning],
        'MAE': [mae_tuning],
        'MAPE': [mape_tuning]
    })

    # Concatenar a nova linha ao DataFrame existente
    metrics_df = pd.concat([metrics_df, new_row], ignore_index=True)

In [48]:
# Valores das métricas de performance para diferentes combinações dos hiper-parâmetros 
metrics_df

,alpha,max_iter,R2,MSE,RMSE,MAE,MAPE
0,0.5,100,0.039927,458.446228,21.411357,17.039612,8.682477
1,0.5,500,0.039927,458.446228,21.411357,17.039612,8.682477
2,0.5,1000,0.039927,458.446228,21.411357,17.039612,8.682477
3,0.5,1500,0.039927,458.446228,21.411357,17.039612,8.682477
4,1.0,100,0.039928,458.445477,21.411340,17.039472,8.682412
...,...,...,...,...,...,...,...
79,10.0,1500,0.039937,458.441098,21.411238,17.037610,8.681342
80,10.5,100,0.039937,458.441309,21.411243,17.037527,8.681287
81,10.5,500,0.039937,458.441309,21.411243,17.037527,8.681287
82,10.5,1000,0.039937,458.441309,21.411243,17.037527,8.681287


Em particular, estamos interessados em buscar o conjunto de hiper-parâmetros que minimize a **Raiz do Erro Quadrático Médio (RMSE)**. 

In [49]:
# exibe as 5 primeiras linhas do dataset ordenado do menor para o maior RMSE
metrics_df.sort_values(['RMSE'], ascending=True)[:5]

,alpha,max_iter,R2,MSE,RMSE,MAE,MAPE
62,8.0,1000,0.039938,458.440703,21.411228,17.037961,8.681568
60,8.0,100,0.039938,458.440703,21.411228,17.037961,8.681568
61,8.0,500,0.039938,458.440703,21.411228,17.037961,8.681568
63,8.0,1500,0.039938,458.440703,21.411228,17.037961,8.681568
56,7.5,100,0.039938,458.440719,21.411229,17.038050,8.681625


Observe que todas as cinco métricas de performance apresentam valores idênticos para as quatro linhas iniciais do dataset **ordenado**. Nesse caso, dado que o valor de `alpha` é igual à `8.0` para todas, convém escolher o menor `max_iter` (`100`), sem prejuízo da performance. 

In [50]:
# Melhores hiper-parâmetros: `alpha = 8.0` e `max_iter = 100`
best_alpha = 8.0
best_max_iter = 100

## 5.4 - Desempenho do modelo para os dados de teste

In [51]:
# definição do modelo
ridge_test = lm.Ridge(alpha=best_alpha, max_iter=best_max_iter, random_state=0)

# Juntar os dados de treino e validação
ridge_test.fit( pd.concat( [X_train, X_val] ),
                pd.concat( [y_train, y_val] ) )

# Predição sobre os dados de teste
y_pred_test = ridge_test.predict( X_test )

# Métricas
df_metrics_test = metrics(y_test, y_pred_test, 'Test', 'Ridge Linear Regression')
df_metrics_test

,Algorithm,Step,R2,MSE,RMSE,MAE,MAPE
0,Ridge Linear Regression,Test,0.0511,461.9991,21.4942,17.1427,8.5366


# 6.0 - Gera as planilhas com os resultados

In [53]:
exporta_excel('linear_ridge_regression')